**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/joins-and-unions).**

---


# Introduction

Here, you'll use different types of SQL **JOINs** to answer questions about the [Stack Overflow](https://www.kaggle.com/stackoverflow/stackoverflow) dataset.

Before you get started, run the following cell to set everything up.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex1 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The code cell below fetches the `posts_questions` table from the `stackoverflow` dataset.  We also preview the first five rows of the table.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "posts_questions" table
table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,67757982,Carry select adder pipelining in vhdl,<p>Please help me how can i pipeline this desi...,NaN,0,0,NaT,2021-05-30 05:26:15.923000+00:00,NaN,2021-05-30 05:26:15.923000+00:00,NaT,None,NaN,None,15758793,None,1,0,vhdl,1
1,67472359,filebeat registry file shows offset to 0,"<p>We have configured filebeat in the server, ...",NaN,0,0,NaT,2021-05-10 14:31:52.090000+00:00,NaN,2021-05-10 14:31:52.090000+00:00,NaT,None,NaN,None,11549576,None,1,0,offset|filebeat,2
2,67485809,Multilayer Graph/Multilayer Network and Graph ...,<p>I'm new to building graphs and GCNs and I w...,NaN,0,0,NaT,2021-05-11 11:26:22.747000+00:00,NaN,2021-05-11 11:26:22.747000+00:00,NaT,None,NaN,None,13916429,None,1,0,graph|neural-network|multi-layer,2
3,67514172,Creating a boxed outline in theme in wordpress,<p>Want to remove the green highlighted area a...,NaN,0,0,NaT,2021-05-13 05:11:07.957000+00:00,NaN,2021-05-13 05:11:07.957000+00:00,NaT,None,NaN,None,15319352,None,1,-1,frontend|wordpress-theming|themes|elementor,2
4,67570205,Regarding the Reshaping the data and Improving...,<p>1.How could I resolve the layer incompatobi...,NaN,0,0,NaT,2021-05-17 13:00:28.623000+00:00,NaN,2021-05-17 13:00:28.623000+00:00,NaT,None,NaN,None,15064017,None,1,0,lstm,2


We also take a look at the `posts_answers` table.

In [3]:
# Construct a reference to the "posts_answers" table
table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,67154495,None,<p>You can get two arrays of characters from t...,None,None,0,NaT,2021-04-18 23:54:41.763000+00:00,None,2021-04-19 00:01:50.707000+00:00,2021-04-19 00:01:50.707000+00:00,user15675591,NaN,user15675591,NaN,64050791,2,0,None,None
1,67154892,None,<p>To check if <em>two strings are anagrams</e...,None,None,0,NaT,2021-04-19 01:18:10.110000+00:00,None,2021-04-19 01:18:10.110000+00:00,NaT,None,NaN,user15675591,NaN,63352574,2,0,None,None
2,67155332,None,<p>You can first get arrays of characters from...,None,None,0,NaT,2021-04-19 02:44:10.033000+00:00,None,2021-04-19 02:59:37.867000+00:00,2021-04-19 02:59:37.867000+00:00,user15675591,NaN,user15675591,NaN,60867630,2,0,None,None
3,67156330,None,<p>You can use <em>Java 8 Stream</em>: <a href...,None,None,0,NaT,2021-04-19 05:21:34.760000+00:00,None,2021-04-25 15:42:47.580000+00:00,2021-04-25 15:42:47.580000+00:00,user15675591,NaN,user15675591,NaN,66065890,2,0,None,None
4,67156874,None,<p>As others have already mentioned that &quot...,None,None,0,NaT,2021-04-19 06:22:17.893000+00:00,None,2021-04-19 06:22:17.893000+00:00,NaT,None,NaN,user10064176,NaN,57751417,2,0,None,None


You will work with both of these tables to answer the questions below.

# Exercises

### 1) How long does it take for questions to receive answers?

You're interested in exploring the data to have a better understanding of how long it generally takes for questions to receive answers.  Armed with this knowledge, you plan to use this information to better design the order in which questions are presented to Stack Overflow users.

With this goal in mind, you write the query below, which focuses on questions asked in January 2018.  It returns a table with two columns:
- `q_id` - the ID of the question
- `time_to_answer` - how long it took (in seconds) for the question to receive an answer

Run the query below (without changes), and take a look at the output.

In [4]:
first_query = """
              SELECT q.id AS q_id,
                  MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) as time_to_answer
              FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                  INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
              ON q.id = a.parent_id
              WHERE q.creation_date >= '2018-01-01' and q.creation_date < '2018-02-01'
              GROUP BY q_id
              ORDER BY time_to_answer
              """

first_result = client.query(first_query).result().to_dataframe()
print("Percentage of answered questions: %s%%" % \
      (sum(first_result["time_to_answer"].notnull()) / len(first_result) * 100))
print("Number of questions:", len(first_result))
first_result.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Percentage of answered questions: 100.0%
Number of questions: 134648


,q_id,time_to_answer
0,48054907,0
1,48450561,0
2,48396661,0
3,48102324,0
4,48121252,0


You're surprised at the results and strongly suspect that something is wrong with your query.  In particular,
- According to the query, 100% of the questions from January 2018 received an answer.  But, you know that ~80% of the questions on the site usually receive an answer.
- The total number of questions is surprisingly low.  You expected to see at least 150,000 questions represented in the table.

Given these observations, you think that the type of **JOIN** you have chosen has inadvertently excluded unanswered questions.  Using the code cell below, can you figure out what type of **JOIN** to use to fix the problem so that the table includes unanswered questions?

**Note**: You need only amend the type of **JOIN** (i.e., **INNER**, **LEFT**, **RIGHT**, or **FULL**) to answer the question successfully.

In [5]:
# Your code here
correct_query = """
                SELECT q.id AS q_id,
                  MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) as time_to_answer
              FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                  FULL JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
              ON q.id = a.parent_id
              WHERE q.creation_date >= '2018-01-01' and q.creation_date < '2018-02-01'
              GROUP BY q_id
              ORDER BY time_to_answer
                """

# Check your answer
q_1.check()

# Run the query, and return a pandas DataFrame
correct_result = client.query(correct_query).result().to_dataframe()
print("Percentage of answered questions: %s%%" % \
      (sum(correct_result["time_to_answer"].notnull()) / len(correct_result) * 100))
print("Number of questions:", len(correct_result))

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,q_id,time_to_answer
0,48550160,NaN
1,48539616,NaN
2,48071583,NaN
3,48497136,NaN
4,48320915,NaN


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Percentage of answered questions: 83.08578974324166%
Number of questions: 162059


In [6]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Initial questions and answers, Part 1

You're interested in understanding the initial experiences that users typically have with the Stack Overflow website.  Is it more common for users to first ask questions or provide answers?  After signing up, how long does it take for users to first interact with the website?  To explore this further, you draft the (partial) query in the code cell below.

The query returns a table with three columns:
- `owner_user_id` - the user ID
- `q_creation_date` - the first time the user asked a question 
- `a_creation_date` - the first time the user contributed an answer 

You want to keep track of users who have asked questions, but have yet to provide answers.  And, your table should also include users who have answered questions, but have yet to pose their own questions.  

With this in mind, please fill in the appropriate **JOIN** (i.e., **INNER**, **LEFT**, **RIGHT**, or **FULL**) to return the correct information.  

**Note**: You need only fill in the appropriate **JOIN**.  All other parts of the query should be left as-is.  (You also don't need to write any additional code to run the query, since the `cbeck()` method will take care of this for you.)

To avoid returning too much data, we'll restrict our attention to questions and answers posed in January 2019.  We'll amend the timeframe in Part 2 of this question to be more realistic!

In [7]:
# Your code here
q_and_a_query = """
                SELECT q.owner_user_id AS owner_user_id,
                    MIN(q.creation_date) AS q_creation_date,
                    MIN(a.creation_date) AS a_creation_date
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                    FULL JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                ON q.owner_user_id = a.owner_user_id 
                WHERE q.creation_date >= '2019-01-01' AND q.creation_date < '2019-02-01' 
                    AND a.creation_date >= '2019-01-01' AND a.creation_date < '2019-02-01'
                GROUP BY owner_user_id
                """

# Check your answer
q_2.check()

,owner_user_id,q_creation_date,a_creation_date
0,819355,2019-01-15 11:51:35.547000+00:00,2019-01-01 05:25:31.067000+00:00
1,10228119,2019-01-22 17:53:28.047000+00:00,2019-01-01 08:53:26.943000+00:00
2,1319998,2019-01-06 08:25:43.893000+00:00,2019-01-01 10:33:52.593000+00:00
3,4934640,2019-01-18 23:28:22.627000+00:00,2019-01-01 11:21:55.790000+00:00
4,4511297,2019-01-31 08:38:10.607000+00:00,2019-01-01 13:19:36.403000+00:00


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) Initial questions and answers, Part 2

Now you'll address a more realistic (and complex!) scenario.  To answer this question, you'll need to pull information from *three* different tables!  This syntax very similar to the case when we have to join only two tables.  For instance, consider the three tables below.

![three tables](https://i.imgur.com/OyhYtD1.png)

We can use two different **JOINs** to link together information from all three tables, in a single query.

![double join](https://i.imgur.com/G6buS7P.png)

With this in mind, say you're interested in understanding users who joined the site in January 2019.  You want to track their activity on the site: when did they post their first questions and answers, if ever?

Write a query that returns the following columns:
- `id` - the IDs of all users who created Stack Overflow accounts in January 2019 (January 1, 2019, to January 31, 2019, inclusive)
- `q_creation_date` - the first time the user posted a question on the site; if the user has never posted a question, the value should be null
- `a_creation_date` - the first time the user posted a question on the site; if the user has never posted a question, the value should be null

Note that questions and answers posted after January 31, 2019, should still be included in the results.  And, all users who joined the site in January 2019 should be included (even if they have never posted a question or provided an answer).

The query from the previous question should be a nice starting point to answering this question!  You'll need to use the `posts_answers` and `posts_questions` tables.  You'll also need to use the `users` table from the Stack Overflow dataset.  The relevant columns from the `users` table are `id` (the ID of each user) and `creation_date` (when the user joined the Stack Overflow site, in DATETIME format).

In [9]:
# Construct a reference to the "users" table
table_ref = dataset_ref.table("users")
# API request - fetch the table
table = client.get_table(table_ref)
# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  


,id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,148,Adrian Clark,<p>I'm a Microsoft .Net Developer and work in ...,None,2008-08-02 15:01:32.393000+00:00,2021-05-25 05:59:07.473000+00:00,"Brisbane, Australia",12189,325,26,734,None,None
1,850,Ekkmanz,<p>Rails programmer. Former PHP programmer. Lo...,None,2008-08-09 15:08:16.490000+00:00,2021-05-28 01:49:51.497000+00:00,Thailand,474,41,0,74,None,None
2,1882,Chris Patterson,<p>Chris is an Enterprise Architect and the Se...,None,2008-08-19 03:37:09.173000+00:00,2021-05-30 01:56:33.670000+00:00,"Dallas, TX, USA",16928,153,3,2554,None,http://blog.phatboyg.com/
3,1945,vanhornRF,<p>Interactive Front End Developer</p>,None,2008-08-19 14:49:22.467000+00:00,2018-08-27 16:24:27.637000+00:00,"River Falls, Wisconsin, United States",328,178,2,68,None,https://www.vipfanexperiences.com
4,2016,ftdysa,None,None,2008-08-19 19:49:14.140000+00:00,2020-04-27 15:01:34.933000+00:00,"Atlanta, GA, United States",1233,410,34,190,None,None


In [10]:
# Your code here
three_tables_query = """
                    SELECT u.id AS id,
                        MIN(q.creation_date) AS q_creation_date,
                        MIN(a.creation_date) AS a_creation_date
                    
                    FROM `bigquery-public-data.stackoverflow.users` AS u
                    LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                        ON u.id = a.owner_user_id 
                    LEFT JOIN `bigquery-public-data.stackoverflow.posts_questions` AS q
                        ON u.id = q.owner_user_id 
                        
                    WHERE u.creation_date>='2019-01-01' AND u.creation_date <'2019-02-01'
                    GROUP BY id
                     """

# Check your answer
q_3.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,id,q_creation_date,a_creation_date
0,10947215,2019-01-21 23:06:47.690000+00:00,2021-02-04 16:21:01.267000+00:00
1,10889304,2020-11-06 19:07:26.463000+00:00,2020-08-12 06:13:35.130000+00:00
2,10983510,2019-04-26 12:45:10.920000+00:00,2019-04-26 12:49:38.987000+00:00
3,10934891,2020-02-15 21:07:54.887000+00:00,2021-03-08 07:22:03.677000+00:00
4,10878042,2019-01-07 08:28:57.940000+00:00,NaT


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [11]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

### 4) How many distinct users posted on January 1, 2019?

In the code cell below, write a query that returns a table with a single column:
- `owner_user_id` - the IDs of all users who posted at least one question or answer on January 1, 2019.  Each user ID should appear at most once.

In the `posts_questions` (and `posts_answers`) tables, you can get the ID of the original poster from the `owner_user_id` column.  Likewise, the date of the original posting can be found in the `creation_date` column.  

In order for your answer to be marked correct, your query must use a **UNION**.

In [12]:
# Your code here
all_users_query = """
                  SELECT q.owner_user_id 
                  
                  FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                  WHERE EXTRACT(DATE FROM q.creation_date) = '2019-01-01'
                  UNION DISTINCT
                  
                  SELECT a.owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_answers` AS a
                  WHERE EXTRACT(DATE FROM a.creation_date) = '2019-01-01'
                  """

# Check your answer
q_4.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,owner_user_id
0,4381453.0
1,4457867.0
2,10854019.0
3,10229061.0
4,1174869.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [13]:
# Lines below will give you a hint or solution code
#q_4.hint()
#q_4.solution()

# Keep going

Learn how to use **[analytic functions](https://www.kaggle.com/alexisbcook/analytic-functions)** to perform complex calculations with minimal SQL code.

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161315) to chat with other Learners.*